# Deployment and Monitor

This notebook deploy a model trained in pipeline, and monitor it.

## Update Model Package Approval Status

We can approve the model using the SageMaker Studio UI or programmatically as shown below.

In [8]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [9]:
pipeline_name = 'qa-pipeline-16323001711632300171'

### Retrive Model From Pipelines

In [10]:
import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    #        print('Executions for our pipeline...')
    #        print(pipeline_execution_status)
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

Succeeded
[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16323001711632300171/execution/67ot3yw4s5kf',
  'PipelineExecutionDisplayName': 'nidome',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2021, 9, 22, 9, 27, 48, 609000, tzinfo=tzlocal())},
 {'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16323001711632300171/execution/vqiccs4mntb5',
  'PipelineExecutionDisplayName': 'execution-1632300253350',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2021, 9, 22, 8, 44, 13, 274000, tzinfo=tzlocal())}]


### List Execution Steps

In [11]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

Succeeded


In [12]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16323001711632300171/execution/67ot3yw4s5kf


In [13]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

{'PipelineExecutionSteps': [{'EndTime': datetime.datetime(2021, 9, 22, 9, 37, 58, 980000, tzinfo=tzlocal()),
                             'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:model/pipelines-67ot3yw4s5kf-createqamodel-en4hwj8lbg'}},
                             'StartTime': datetime.datetime(2021, 9, 22, 9, 37, 57, 370000, tzinfo=tzlocal()),
                             'StepName': 'CreateQAModel',
                             'StepStatus': 'Succeeded'},
                            {'EndTime': datetime.datetime(2021, 9, 22, 9, 37, 58, 187000, tzinfo=tzlocal()),
                             'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:model-package/qamodelpackagegroup/7'}},
                             'StartTime': datetime.datetime(2021, 9, 22, 9, 37, 57, 298000, tzinfo=tzlocal()),
                             'StepName': 'QARegisterModel',
                             'StepStatus': 'Succeeded'},
                    

### View Registered Model and Update Model Approval Status

In [19]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "QARegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break
print(model_package_arn)

arn:aws:sagemaker:us-east-1:093729152554:model-package/qamodelpackagegroup/7


In [21]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",  # Other options are Rejected and PendingManualApproval
)

### View Created Model

In [23]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "CreateQAModel":
        model_arn = execution_step["Metadata"]["Model"]["Arn"]
        break
print(model_arn)

created_model_name = model_arn.split("/")[-1]
print('created_model_name:', created_model_name)

arn:aws:sagemaker:us-east-1:093729152554:model/pipelines-67ot3yw4s5kf-createqamodel-en4hwj8lbg
created_model_name: pipelines-67ot3yw4s5kf-createqamodel-en4hwj8lbg


## Create Model Endpoint from Model Registry and Configure It to Capture Requests

### Create model from registry

More details here: https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-deploy.html

In [24]:
import time

timestamp = int(time.time())

model_from_registry_name = "qa-model-from-registry-{}".format(timestamp)
print("Model from registry name : {}".format(model_from_registry_name))

model_registry_package_container = {
    "ModelPackageName": model_package_arn,
}

Model from registry name : qa-model-from-registry-1632465588


In [25]:
from pprint import pprint

create_model_from_registry_respose = sm.create_model(
    ModelName=model_from_registry_name, ExecutionRoleArn=role, PrimaryContainer=model_registry_package_container
)
pprint(create_model_from_registry_respose)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:093729152554:model/qa-model-from-registry-1632465588',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '95',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 24 Sep 2021 06:39:49 GMT',
                                      'x-amzn-requestid': 'b922cc61-ab4e-4cd4-b067-2212e5acfa89'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'b922cc61-ab4e-4cd4-b067-2212e5acfa89',
                      'RetryAttempts': 0}}


In [26]:
model_from_registry_arn = create_model_from_registry_respose["ModelArn"]
model_from_registry_arn

'arn:aws:sagemaker:us-east-1:093729152554:model/qa-model-from-registry-1632465588'

### Configure Endpoint to Capture Data from Requests and Responses

Check API for [create_endpoint_config](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config)

In [27]:
data_capture_bucket = 'sm-nlp-data'
data_capture_prefix = 'inference/'

Creates an endpoint configuration that Amazon SageMaker hosting services uses to deploy models.

In [37]:
endpoint_config_name = "qa-model-from-registry-epc-{}".format(timestamp)
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": created_model_name,
            "VariantName": "AllTraffic",
        }
    ],
    DataCaptureConfig={
        'EnableCapture': True,
        'InitialSamplingPercentage': 100,
        'DestinationS3Uri': f"s3://{data_capture_bucket}/{data_capture_prefix}",
        'CaptureOptions': [
            {
                'CaptureMode': 'Input'
            },
            {
                'CaptureMode': 'Output'
            },
        ]
    }
)

qa-model-from-registry-epc-1632465588


delete an existing config with the following command:

In [36]:
# !aws sagemaker delete-endpoint-config --endpoint-config-name $endpoint_config_name

In [39]:
pipeline_endpoint_name = "qa-model-from-registry-ep-{}".format(timestamp)
print("EndpointName={}".format(pipeline_endpoint_name))

create_endpoint_response = sm.create_endpoint(
    EndpointName=pipeline_endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

EndpointName=qa-model-from-registry-ep-1632465588
arn:aws:sagemaker:us-east-1:093729152554:endpoint/qa-model-from-registry-ep-1632465588


In [40]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, pipeline_endpoint_name
        )
    )
)

In [41]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=pipeline_endpoint_name)

CPU times: user 182 ms, sys: 19.2 ms, total: 202 ms
Wall time: 8min 31s


### List All Artifacts

In [42]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

{'StepName': 'Processing', 'StartTime': datetime.datetime(2021, 9, 22, 9, 27, 49, 711000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 9, 22, 9, 32, 16, 867000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:processing-job/pipelines-67ot3yw4s5kf-processing-jtj2dbws96'}}}
pipelines-67ot3yw4s5kf-processing-jtj2dbws96


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...22-08-43-33-756/input/code/preprocess.py,Input,DataSet,ContributedTo,artifact
1,s3://sm-nlp-data/nlu/data/qa_raw.zip,Input,DataSet,ContributedTo,artifact
2,68331...om/sagemaker-scikit-learn:0.23-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://sm-nlp-data/nlu/data/processed/,Output,DataSet,Produced,artifact


{'StepName': 'Train', 'StartTime': datetime.datetime(2021, 9, 22, 9, 32, 17, 256000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 9, 22, 9, 32, 17, 630000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'CacheHitResult': {'SourcePipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16323001711632300171/execution/vqiccs4mntb5'}, 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:training-job/pipelines-vqiccs4mntb5-train-jdmj5liqyt'}}}
pipelines-vqiccs4mntb5-train-jdmj5liqyt


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://sm-nlp-data/nlu/data/processed/,Input,DataSet,ContributedTo,artifact
1,76310...onaws.com/pytorch-training:1.8.1-gpu-py3,Input,Image,ContributedTo,artifact
2,s3://...tb5-Train-JDmj5LiQyt/output/model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'EvaluateModel', 'StartTime': datetime.datetime(2021, 9, 22, 9, 32, 18, 202000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 9, 22, 9, 37, 55, 910000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:processing-job/pipelines-67ot3yw4s5kf-evaluatemodel-d7wij9sxb2'}}}


...

None

{'StepName': 'QARegisterModel', 'StartTime': datetime.datetime(2021, 9, 22, 9, 37, 57, 298000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 9, 22, 9, 37, 58, 187000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:model-package/qamodelpackagegroup/7'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,qamodelpackagegroup-7-Approved-1632311055-aws-...,Input,Approval,ContributedTo,action
1,s3://...tb5-Train-JDmj5LiQyt/output/model.tar.gz,Input,Model,ContributedTo,artifact
2,76310...naws.com/pytorch-inference:1.8.1-gpu-py3,Input,Image,ContributedTo,artifact
3,qamodelpackagegroup-7-PendingManualApproval-16...,Input,Approval,ContributedTo,action
4,QAModelPackageGroup-1631002331-aws-model-packa...,Output,ModelGroup,AssociatedWith,context


{'StepName': 'CreateQAModel', 'StartTime': datetime.datetime(2021, 9, 22, 9, 37, 57, 370000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 9, 22, 9, 37, 58, 980000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:model/pipelines-67ot3yw4s5kf-createqamodel-en4hwj8lbg'}}}


None

### Test the Deployed Model

CSVSerializer: [DOC](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html#sagemaker.serializers.CSVSerializer) </br>
JSONDeserializer: [DOC](https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#sagemaker.deserializers.JSONDeserializer)

In [43]:
import json
from sagemaker.pytorch.model import PyTorchPredictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = PyTorchPredictor(
    endpoint_name=pipeline_endpoint_name,
    sagemaker_session=sess,
    serializer=CSVSerializer(),
    deserializer=JSONDeserializer(),
)

In [54]:
!touch processed/psuedo/seq.in

In [82]:
with open('processed/psuedo/seq.in', 'r') as f:
    lines = f.read()
    predicted = predictor.predict(lines)
predicted

{'text': [['伊'],
  ['坂'],
  ['幸'],
  ['太'],
  ['郎'],
  ['写'],
  ['了'],
  ['哪'],
  ['些'],
  ['书'],
  [],
  ['O'],
  ['N'],
  ['E'],
  [],
  ['P'],
  ['I'],
  ['E'],
  ['C'],
  ['E'],
  ['総'],
  ['集'],
  ['编'],
  [],
  ['T'],
  ['H'],
  ['E'],
  [],
  ['F'],
  ['I'],
  ['R'],
  ['S'],
  ['T'],
  [],
  ['L'],
  ['O'],
  ['G'],
  ['是'],
  ['谁'],
  ['写'],
  ['的'],
  [],
  ['高'],
  ['效'],
  ['管'],
  ['理'],
  ['W'],
  ['i'],
  ['n'],
  ['d'],
  ['o'],
  ['w'],
  ['s'],
  ['网'],
  ['络'],
  ['/'],
  ['W'],
  ['i'],
  ['n'],
  ['3'],
  ['2'],
  [],
  ['P'],
  ['e'],
  ['r'],
  ['l'],
  ['应'],
  ['用'],
  ['之'],
  ['道'],
  ['的'],
  ['作'],
  ['者'],
  ['是'],
  ['谁'],
  [],
  ['D'],
  ['a'],
  ['v'],
  ['e'],
  ['写'],
  ['了'],
  ['什'],
  ['么'],
  ['书'],
  [],
  ['洪'],
  ['荒'],
  ['之'],
  ['武'],
  ['道'],
  ['是'],
  ['谁'],
  ['写'],
  ['的'],
  [],
  ['玄'],
  ['黄'],
  ['真'],
  ['人'],
  ['写'],
  ['了'],
  ['哪'],
  ['些'],
  ['书'],
  [],
  ['风'],
  ['景'],
  ['景'],
  ['观'],
  ['工'],
  ['程'],
  ['体'],
  ['系'],

### View Captured Data

In [62]:
s3 = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}{}'.format(data_capture_prefix, pipeline_endpoint_name)
result = s3.list_objects(Bucket=data_capture_bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
inference/qa-model-from-registry-ep-1632465588/AllTraffic/2021/09/24/07/27-23-527-83199569-6b92-41b3-8074-7ef23d55164a.jsonl


In [66]:
capture_files[0]

'inference/qa-model-from-registry-ep-1632465588/AllTraffic/2021/09/24/07/27-23-527-83199569-6b92-41b3-8074-7ef23d55164a.jsonl'

use the S3Downloader utility to view and download the captured data in Amazon S3:

In [70]:
from sagemaker.s3 import S3Downloader
traffic = S3Downloader.read_file(f"s3://{data_capture_bucket}/{capture_files[0]}")
traffic

'{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"5LyK5Z2C5bm45aSq6YOO5YaZ5LqG5ZOq5Lqb5LmmCgpPTkUgUElFQ0Xnt4/pm4bnvJYgVEhFIEZJUlNUIExPR+aYr+iwgeWGmeeahAoK6auY5pWI566h55CGV2luZG93c+e9kee7nC9XaW4zMiBQZXJs5bqU55So5LmL6YGT55qE5L2c6ICF5piv6LCBCgpEYXZl5YaZ5LqG5LuA5LmI5LmmCgrmtKrojZLkuYvmrabpgZPmmK/osIHlhpnnmoQKCueOhOm7hOecn+S6uuWGmeS6huWTquS6m+S5pgoK6aOO5pmv5pmv6KeC5bel56iL5L2T57O75YyW5piv6LCB55qE5L2c5ZOBCgrlvq7nn6XmsYfvvJrkuIfniannroDlj7LnmoTkvZzogIXmmK/osIEKCuWtvemYs+eahOS9nOiAheaYr+iwgQoK6IyF5pyI5YaZ5LqG5ZOq5Lqb5LmmCgrmnKrmnaXlqLHkuZDns7vnu5/mmK/osIHlhpnnmoQKCuWwj+WDp+S4jeaVsuacqOmxvOacieS7gOS5iOiRl+S9nAoK6YeR6KOF5Zub5aSn5omN5a2Q5piv6LCB55qE5L2c5ZOBCgrnvZfmsLjotKTlr7zmvJTkuoblk6rkupvnlLXlvbEKCuS4uuS6huS9oOaIkeaEv+aEj+eDreeIseaVtOS4quS4lueVjOaYr+iwgeeahOS9nOWTgQoK6YOt6JmO5a+85ryU5LqG5ZOq5Lqb55S15b2xCgrkuLrkuobkvaDmiJHmhL/mhI/ng63niLHmlbTkuKrkuJbnlYzmmK/osIHlr7zmvJTnmoQKCueBree9quW4iOeahOWvvOa8lOaYr+iwgQoK5p2o6IuX5a+85ryU5LqG5ZOq5Lqb55S16KeG5YmnCgrnga

In [74]:
endpoint_input_data = json.loads(traffic)['captureData']['endpointInput']['data']
endpoint_output_data = json.loads(traffic)['captureData']['endpointOutput']['data']

Decode payload with base64 library

In [80]:
import base64

base64.b64decode(endpoint_input_data).decode('utf-8')

'伊坂幸太郎写了哪些书\n\nONE PIECE総集编 THE FIRST LOG是谁写的\n\n高效管理Windows网络/Win32 Perl应用之道的作者是谁\n\nDave写了什么书\n\n洪荒之武道是谁写的\n\n玄黄真人写了哪些书\n\n风景景观工程体系化是谁的作品\n\n微知汇：万物简史的作者是谁\n\n孽阳的作者是谁\n\n茅月写了哪些书\n\n未来娱乐系统是谁写的\n\n小僧不敲木鱼有什么著作\n\n金装四大才子是谁的作品\n\n罗永贤导演了哪些电影\n\n为了你我愿意热爱整个世界是谁的作品\n\n郭虎导演了哪些电影\n\n为了你我愿意热爱整个世界是谁导演的\n\n灭罪师的导演是谁\n\n杨苗导演了哪些电视剧\n\n灭罪师是谁导演的\n\n五百导演了哪些电视剧\n\n穆念慈的丈夫是谁\n\n杨康的配偶是谁'

In [81]:
base64.b64decode(endpoint_output_data).decode('utf-8')

'{"text": [["伊"], ["坂"], ["幸"], ["太"], ["郎"], ["写"], ["了"], ["哪"], ["些"], ["书"], [], [], ["O"], ["N"], ["E"], [], ["P"], ["I"], ["E"], ["C"], ["E"], ["総"], ["集"], ["编"], [], ["T"], ["H"], ["E"], [], ["F"], ["I"], ["R"], ["S"], ["T"], [], ["L"], ["O"], ["G"], ["是"], ["谁"], ["写"], ["的"], [], [], ["高"], ["效"], ["管"], ["理"], ["W"], ["i"], ["n"], ["d"], ["o"], ["w"], ["s"], ["网"], ["络"], ["/"], ["W"], ["i"], ["n"], ["3"], ["2"], [], ["P"], ["e"], ["r"], ["l"], ["应"], ["用"], ["之"], ["道"], ["的"], ["作"], ["者"], ["是"], ["谁"], [], [], ["D"], ["a"], ["v"], ["e"], ["写"], ["了"], ["什"], ["么"], ["书"], [], [], ["洪"], ["荒"], ["之"], ["武"], ["道"], ["是"], ["谁"], ["写"], ["的"], [], [], ["玄"], ["黄"], ["真"], ["人"], ["写"], ["了"], ["哪"], ["些"], ["书"], [], [], ["风"], ["景"], ["景"], ["观"], ["工"], ["程"], ["体"], ["系"], ["化"], ["是"], ["谁"], ["的"], ["作"], ["品"], [], [], ["微"], ["知"], ["汇"], ["："], ["万"], ["物"], ["简"], ["史"], ["的"], ["作"], ["者"], ["是"], ["谁"], [], [], ["孽"], ["阳"], ["的"], ["作"], ["者"], ["是"], ["谁"], []

## Monitor SageMaker endpoints

There are mainly data quality monitoring and model quality monitoring, in which:

- data quality monitoring captures inference input, and compares data statistics like min, max with a baseline created from dataset [[Monitor Data Quality](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-quality.html)]
- model quality monitoring monitors the performance of a model by comparing the predictions that the model makes with the actual ground truth labels that the model attempts to predict. [[Monitor Model Quality](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality.html)]

Data quality is only applicapable for tabular data, therefore **not suitable** for this question understanding use case. Here we implement a quality monitoring for model quality.

Reference:
- AWS Doc: [Amazon SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html)
- SageMaker Doc: [Amazon SageMaker Model Monitor](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_monitoring.html)
- [AWS Workshop: Model Monitor](https://sagemaker-immersionday.workshop.aws/lab4/monitoring.html)
- [Create a Model Quality Baseline](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-baseline.html)

### Create a Model Quality Baseline

1.  Create an instance of the ModelQualityMonitor class. 

Check SageMaker ModelQualityMonitor API: [Doc](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html)

In [115]:
baseline_output_bucket = 'sm-nlp-data'
baseline_job_name = "QABaseLineJob"

In [89]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

In [86]:
model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.4xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=sess
)

2. Create a baseline dataset in JSON with validation data

In [114]:
with open('processed/dev/seq.in') as f:
    x_input = f.readlines()
    x_input = [x.strip() for x in x_input]
with open('processed/dev/label') as f:
    y_output = f.readlines()
    y_output = [y.strip() for y in y_output]
with open('processed/dev/seq.out') as f:
    seq_output = f.readlines()
    seq_output = [seq.strip().split() for seq in seq_output]
val_dataset = {
    'seq_in': x_input,
    'seq_out': seq_output,
    'label': y_output
}
with open('val_dataset.json', 'w') as f:
    json.dump(val_dataset, f, ensure_ascii=False)

3. Now call the suggest_baseline method of the ModelQualityMonitor object to run a baseline job. We need a baseline dataset that contains both predictions and labels stored in Amazon S3.

In [ ]:
job = model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset='./val_dataset.json', # The S3 location of the validation dataset.
    dataset_format=DatasetFormat.json(lines=False), # Whether the file should be read as a json object per line
    output_s3_uri = f"s3://{baseline_output_bucket}/{baseline_job_name}/", # The S3 location to store the results.
    problem_type='MulticlassClassification',
    inference_attribute= "label", # The column in the dataset that contains predictions.
    ground_truth_attribute= "label" # The column in the dataset that contains ground truth labels.
)

In [118]:
job.wait(logs=True)

2021-09-24 09:08:07,412 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:093729152554:processing-job/qabaselinejob', 'ProcessingJobName': 'QABaseLineJob', 'Environment': {'analysis_type': 'MODEL_QUALITY', 'dataset_format': '{"json": {"lines": false}}', 'dataset_source': '/opt/ml/processing/input/baseline_dataset_input', 'ground_truth_attribute': 'label', 'inference_attribute': 'label', 'output_path': '/opt/ml/processing/output', 'problem_type': 'MulticlassClassification', 'publish_cloudwatch_metrics': 'Disabled'}, 'AppSpecification': {'ImageUri': '156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer', 'ContainerEntrypoint': None, 'ContainerArguments': None}, 'ProcessingInputs': [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3Uri': 's3://sagemaker-us-east-1-093729152554/model-monitor/baselining/QABaseLineJob/input/baseline_dataset_input', 

### Schedule Model Quality Monitoring Jobs 

### Ingest Ground Truth Labels and Merge Them With Predictions 